In [1]:
from torch import cuda

device = 'cuda' if cuda.is_available() else 'cpu'

In [2]:
!git clone https://github.com/xxbean/pozalabs

Cloning into 'pozalabs'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 103 (delta 6), reused 0 (delta 0), pack-reused 69
Receiving objects: 100% (103/103), 64.63 MiB | 17.23 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [3]:

import sys
sys.path.append("/content/pozalabs/ComMU-code")
sys.path.append("/content/pozalabs/Multi-Scale-1D-ResNet")
from model.multi_scale_ori import *
from model.meta_labeling import * #여기에 메타 데이터 함수 정보 있음
import torch
import numpy as np
from torchsummary import summary
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import tqdm
import torch.nn.functional as F
import sys
import glob
import copy

In [5]:
%cd /content/pozalabs/
%ls

/content/pozalabs
CAS_colab.ipynb  ComMU-code/  Multi-Scale-1D-ResNet/  README.md


In [9]:
import glob
from sklearn.model_selection import train_test_split

raw_train = glob.glob("./ComMU-code/dataset/commu_midi/train/raw/**")
raw_val = glob.glob("./ComMU-code/dataset/commu_midi/val/raw/**")

train_meta_npy_ = np.load("/content/pozalabs/ComMU-code/dataset/output_npy_ 2/input_train.npy", allow_pickle=True)[:10000]
train_midi_npy_ = np.load("/content/pozalabs/ComMU-code/dataset/output_npy_ 2/target_train.npy", allow_pickle=True)[:10000]
val_meta_npy_ = np.load("/content/pozalabs/ComMU-code/dataset/output_npy_ 2/input_train.npy", allow_pickle=True)[10000:]
val_midi_npy_ = np.load("/content/pozalabs/ComMU-code/dataset/output_npy_ 2/target_train.npy", allow_pickle=True)[10000:]
test_meta_npy_ = np.load('/content/pozalabs/ComMU-code/dataset/output_npy_ 2/input_val.npy', allow_pickle=True)
test_midi_npy_ = np.load("/content/pozalabs/ComMU-code/dataset/output_npy_ 2/target_val.npy", allow_pickle=True)

label_list = np.unique(train_meta_npy_[:,0])

In [10]:
print(
    
len(train_meta_npy_),
len(train_midi_npy_),
len(val_meta_npy_),
len(val_midi_npy_),
len(test_meta_npy_),
len(test_midi_npy_))

10000 10000 381 381 763 763


In [11]:
class Commu(Dataset):
    def __init__(self, meta_npy, midi_npy,seq_len):
        self.meta_npy = meta_npy
        self.midi_npy = midi_npy
        self.seq_len = seq_len
        self.label_npy = np.zeros_like(self.meta_npy)
        for i in range(11):
            self.label_npy[:,i] = np.array(list(map(meta_list[i],meta_npy[:,i])))

    def __len__(self):
        return len(self.meta_npy)

    def __getitem__(self, idx):
        label = self.label_npy[idx]
        midi = np.zeros((1,self.seq_len))
        midi_real = self.midi_npy[idx][:self.seq_len]
        midi[:,:len(midi_real)] = midi_real
        midi = torch.tensor(midi.tolist(),dtype=torch.float)
        label = torch.tensor(label.tolist(),dtype=torch.float)
        return midi,label

In [12]:
real_data = Commu(train_meta_npy_, train_midi_npy_,512)
real_loader = DataLoader(real_data, batch_size=256, shuffle=True)
val_data = Commu(val_meta_npy_, val_midi_npy_,512)
val_loader = DataLoader(val_data, batch_size=256, shuffle=True)
test_data = Commu(test_meta_npy_, test_midi_npy_,512)
test_loader = DataLoader(test_data, batch_size=256, shuffle=True)

In [13]:
test_batch = next(iter(real_loader))[0]
test_target = next(iter(real_loader))[1]

In [14]:
msresnet = MSResNet(input_channel=1, layers=[1, 1, 1, 1], num_classes=5)
msresnet = msresnet.cuda()
# summary(msresnet,(1,512))

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(msresnet.parameters(), lr=0.001, momentum=0.9)
num_epochs = 100
meta_num = 1

In [18]:
best_acc=0

for epoch in range(num_epochs):  

    running_loss = 0.0
    for i, data in enumerate(tqdm.tqdm(real_loader)):
        msresnet.train()
        # get the inputs; data is a list of [inputs, labels]
        midi,label = data
        midi = midi.cuda()
        label = label.cuda()[:,meta_num]
#         label = F.one_hot(label.to(torch.int64), num_classes = 10)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = msresnet(midi)[0]
        loss = criterion(outputs, label.long())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
    print(running_loss / 41)
    if epoch %6 == 5:
        val_num = 0
        correct_num = 0
        with torch.no_grad():
            msresnet.eval()
            for i, data in enumerate(val_loader):
                midi,label = data
                midi = midi.cuda()
                label = label[:,meta_num]
                num = label.shape[0]
                val_num += num
                outputs = msresnet(midi)[0].argmax(1).cpu()
                correct_num += sum(outputs==label)
                acc = correct_num/val_num
                if acc > best_acc:
                  torch.save(msresnet.state_dict(), '/content/pozalabs/ComMU-code/model_state_dict.pt')
                  best_model = copy.deepcopy(msresnet.state_dict())
                
            print(f"validation accuracy for epoch {epoch} is : {correct_num/val_num}")
        

print('Finished Training')

100%|██████████| 40/40 [00:09<00:00,  4.02it/s]


0.7477613120544248


100%|██████████| 40/40 [00:02<00:00, 18.61it/s]


0.598884576704444


100%|██████████| 40/40 [00:02<00:00, 18.06it/s]


...

0.008543537577568757


100%|██████████| 40/40 [00:02<00:00, 17.33it/s]


0.007871362908811467


100%|██████████| 40/40 [00:02<00:00, 17.23it/s]

0.008399380439120094
Finished Training


In [19]:
msresnet_test = MSResNet(input_channel=1, layers=[1, 1, 1, 1], num_classes=5)
msresnet_test = msresnet_test.cuda()
msresnet_test.load_state_dict(best_model)
msresnet_test.eval()


MSResNet(
  (conv1): Conv1d(1, 64, kernel_size=(7,), stride=(2,), padding=(3,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer3x3_1): Sequential(
    (0): BasicBlock3x3(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(2,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Sequential(
        (0): Conv1d(64, 64, kernel_size=(1,), stride=(2,), bias=False)
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
  )
  (layer3x3_2): Sequential(
    (0): BasicB

In [20]:

correct_test = 0
for i, data in enumerate(test_loader):
    with torch.no_grad():
        midi,label = data
        midi = midi.cuda()
        label = label[:,meta_num]
        num = label.shape[0]
        val_num += num
        outputs = msresnet(midi)[0].argmax(1).cpu()
        correct_num += sum(outputs==label)
        acc = correct_num/val_num
print("Test accuracy:", (100 * float(acc)))


Test accuracy: 81.2062919139862
